# __Создание базы данных MySQL на сервере Linux__

<hr>

В этом проекте используется сервер с операционной системой Ubunty 22.04 LTS. Для работы с MySQL необходимо установить MySQL server на наш сервер прописав команду:   

<code>sudo mysql_secure_installation utility</code>

 Далее необходимо сделать MySQL фоновым сервисом для сервера, это необходимо для непрерывной работы MySQL server:
   
<code>sudo systemctl start mysql</code>

<code>sudo systemctl enable mysql</code>

 Перезагружаем MySQL:
 
<code>sudo systemctl restart mysql</code>

 Теперь добавим простейший FireWall для безопасности:
 
<code>sudo ufw enable</code>
    
 Разрешаем работу MySQL при включенном UFW:
 
<code>sudo ufw allow mysq</code>
 
 Так как нам необходимо настроить удаленный доступ для пользователей MySQL, нужно поправить конфигурационный файл mysqld.cnf. Перейдем в директорию, где он находится:  
 
 
<code>cd /etc/mysql/mysql.conf.d</code>
    
 Откроем файл mysqld.cnf в текстовом редакторе:
 
<code>nano mysqld.cnf</code>
    
 В файле меняем значение строк __bind-address__ и __mysql-bind-address__ на __0.0.0.0__ Сохраняем файл Ctrl+S и закрываем редактор Ctrl+X.
 Заходим в MySQL, как root пользователь:
 
<code>mysql -u root -p</code>
 
 Создадим базу данных с которой будем работать:
 
<code>CREATE DATABASE Название_Базы_Данных;</code> 
 
 А теперь создаем пользователя:

<code>CREATE USER логин@ipV4 IDENTIFIED BY "пароль";</code>

 Но, так как нам необходимо предоставить доступ пользователям из разных точек мы заменем ipV4 на __'%'__. Мы воспользуемся таким запросом:

<code>CREATE USER main@'%' IDENTIFIED BY "P@ssw0rd";</code>

 Предоставим этому пользователю все возможности для работы со всеми таблицами:

<code>GRANT ALL ON testDB.* TO main@'%';</code>
    
 Для просмотра пользователей можно воспользоваться командой:
 
<code>SELECT user, host FROM mysql.user;</code>

 
![Пользователи MySQL](Images/users.png)

 Теперь перейдем к работе с фалами и созданию таблиц!
 <hr>

# __Работа с файлами и автоматизация__

<hr>
У нас есть два файла:
<ul>
    <li><a href="ozon_seller_analytics_results.json">Файл, который мы получили через Ozon Seller API</a></li>
    <li><a href="ozon_seller_glossary.csv">Файл с дополнительной информацией о товарах</a></li>
</ul>

Начнем с работы с более простого. Созданиeм первый DataFrame на его основе. Для этого воспользуемся __Pandas__:

<code>pip install pandas</code>

In [1]:
import pandas as pd

gl = pd.read_csv("Data/ozon_seller_glossary.csv")
gl

,sku,name_short,brand,category,great_category
0,539070047,"Refill ""Лимон"" для мытья посуды 1 л",SYNERGETIC,Посуда,Clean
1,538993940,"Кондиционер бальзам 3,75л Цветущий апельсин и ...",SYNERGETIC,Кондиционеры,Home
2,538862456,"Кондиционер бальзам 3,75л Нежный ирис и гранат",SYNERGETIC,Кондиционеры,Home
3,357690113,"Подгузники размер 2 / MINI, 50шт",SYNERGETIC,Подгузники,Baby
4,357690726,"Подгузники размер 3 / MIDI, 49шт",SYNERGETIC,Подгузники,Baby
...,...,...,...,...,...
865,1099910458,"Набор подгузники Pure&Nature, размер 4 / MAXI,...",SYNERGETIC,Наборы,Baby
866,1100819519,Набор для стирки белья Гель универсальный для ...,SYNERGETIC,Home_Наборы,Home
867,1100871562,Набор для стирки детского белья гель 5л + конд...,SYNERGETIC,Home_Наборы,Home
868,1117832231,"Набор-Капсулы для стирки Color ""40+40""",SYNERGETIC,Home_Наборы,Home


Создадим второй DataFrame. Это будет результирующая таблица, с колонками как в референсе, но с одним дополнительным столбцом __ozon_id__. Ниже референсная таблица:

![Референсная таблица](Images/ref_table.png)

In [2]:
tableColumns = ["ozon_ID", "Направление", "Категория", "sku", "Название товара", "session_view_search", "ordered_units", "revenue", "position_category"]
df = pd.DataFrame(columns=tableColumns)

Начнем работу с __.json файлом__

In [3]:
import json

with open("Data/ozon_seller_analytics_results.json", encoding="utf-8") as file:
    js = json.load(file)

inner = js[0]['result']['data']

Му получили объект List с нужными нам данными. Теперь нам необходимо извлечь данные и поместить их DataFrame, НО перед нами становится задача сопоставить названия товаров из объявлений Ozon (__.json файл__) и файла с кодами __sku__ для правильного заполнения таблицы.

_Пример задачи сопоставления._ В файле с __sku__ один товар называется: 

    Набор для стирки детского белья гель 5л + кондиционер Нежное прикосновение 5л

А в __.json файл__ этот же предмет называется двумя разными способами:
    
    Набор для стирки детского белья SYNERGETIC гель 5л + кондиционер Нежное прикосновение 5л + подарок. Уцененный товар
    Набор для стирки детского белья SYNERGETIC гель 5л + кондиционер Нежное прикосновение 5л + подарок Уцененный товар

И это только один из примеров. Перебрать их все и написать правила для обработки каждого товара невозможно. Из готовых решений существует инструмент для сравнения строк __fuzzywuzzy__, библиотека для Python. Этот инструмент использует метрику _Расстояние Левенштейна_, поэтому для ускорения его работы необходимо дополнительно установить еще один пакет:

<code>pip install fuzzywuzzy</code>

<code>pip install python-Levenshtein</code>

Посмотрим, как он отработает с нашей задачей:

In [4]:
from fuzzywuzzy import fuzz
value1 = "Набор для стирки детского белья гель 5л + кондиционер Нежное прикосновение 5л"
value2 = "Набор для стирки детского белья SYNERGETIC гель 5л + кондиционер Нежное прикосновение 5л + подарок. Уцененный товар"

# Используйте функцию fuzz.ratio() для сравнения строк
similarity_ratio = fuzz.ratio(value1, value2)

print(f"Схожесть: {similarity_ratio}%")

Схожесть: 80%


Мы можем увеличить коэффициент схожести убрав часть строки "Уцененный товар" и точки, так как они есть не в каждой записи и не являются важной частью строки.

In [5]:
value1 = "Набор для стирки детского белья гель 5л + кондиционер Нежное прикосновение 5л"
value2 = "Набор для стирки детского белья SYNERGETIC гель 5л + кондиционер Нежное прикосновение 5л + подарок"

# Используйте функцию fuzz.ratio() для сравнения строк
similarity_ratio = fuzz.ratio(value1, value2)

print(f"Схожесть: {similarity_ratio}%")

Схожесть: 88%


Теперь необходимо найти коэффициент, который будет находить наши названия без ошибок. Мы попробовали разные значения, но при маленьком значение <70 результат никуда не годится, а при большом >80 маленькое количество записей.

<table>
    <tr>
        <th>коэффициент</th>
        <th>записей</th>
    </tr>
    <tr>
        <td>75</td>
        <td>256</td>
    </tr>
    <tr>
        <td>80</td>
        <td>356</td>
    </tr>
    <tr>
        <td>85</td>
        <td>202</td>
    </tr>
</table>

В коде ниже можно менять значения __newCof__ в условии if для изменения приемлемого коэффициент и значения __i__ для ограничения вывода записей (это необходимо для упрощения читаемости файла).

In [6]:
import re

patt = re.compile(r"Уцененный товар|\.")

# количество выводимых значений
i = 25

for item in inner:
    if i == 0:
        break;
    cof = newCof = 0
    for name in gl["name_short"]:
        formattedLine = re.sub(patt,"",item['dimensions'][0]["name"])
        newCof = fuzz.ratio(name, formattedLine)
        
        # Коэффициент меняеться тут newCof
        
        if newCof > cof and newCof >= 75:
            cof = newCof
            factName = name      
    if cof == 0:
        continue
    i -= 1
    print(f"Схожесть: {cof}")
    print(factName)
    print(formattedLine)

Схожесть: 75
"Интенсивное увлажнение и блеск" шампунь + бальзам
Набор натуральный уход для волос "Интенсивное увлажнение и блеск" шампунь + бальзам 
Схожесть: 80
"Максимальное питание и восстановление" шампунь + бальзам
Натуральный уход для волос "Максимальное питание и восстановление" шампунь + бальзам 
Схожесть: 93
Набор для уборки Чистота и блеск, антижир, средство для уборки дома, сантехники 6шт.*500мл.
Synergetic Набор для уборки Чистота и блеск, антижир, средство для уборки дома, сантехники 6шт*500мл 
Схожесть: 93
Набор для уборки Чистота и блеск, антижир, средство для уборки дома, сантехники 6шт.*500мл.
Synergetic Набор для уборки Чистота и блеск, антижир, средство для уборки дома, сантехники 6шт*500мл 
Схожесть: 93
Набор для уборки Чистота и блеск, антижир, средство для уборки дома, сантехники 6шт.*500мл.
Synergetic Набор для уборки Чистота и блеск, антижир, средство для уборки дома, сантехники 6шт*500мл 
Схожесть: 93
Набор для уборки Чистота и блеск, антижир, средство для убор

Если поиграться с коэффициентом можно добиться неплохого результата, __НО__ этого недостаточно, так как с большим коэффициент за 80 единиц иногда проходят ошибки, где предмет определяется правильно, но его объем может отличаться. Так же можно заметить, что все слова из таблицы с кодами __sku__ будут в названии товара __.json файла__, но у второго могут быть дополнительные слова в названии, которые дополняют название товара, но для нас они не важны. Поэтому мы можем сравнивать, строки следующим образом, если все слова из первой строки входят во вторую, то мы считаем, что это одинаковые товары.

In [7]:
def find_right_name(line):
    
    listOfWords = re.sub(patt,"",line).split()
    set1 = set(listOfWords)
    
    for name in gl["name_short"]:
        TableOfWords = name.replace(".","").split()
        set2 = set(TableOfWords)
        
        if set2.issubset(set1):
            return(name)
        
    return False

Если мы применим данный подход на наших значениям. Мы получим следующую таблицу, в которой больше записей, чем при оптимальных коэффициент предыдущего метода, а также они более точные.

In [8]:
for item in inner:
    result = find_right_name(item['dimensions'][0]["name"])
    if result == False:
        continue
    elif  (sum(item["metrics"][0:-3])+sum(item["metrics"][-2:]) > 0) & (int(item["metrics"][8]) == 0):
        continue
    elif  (sum(item["metrics"][0:-3])+sum(item["metrics"][-2:]) == 0) & (int(item["metrics"][8]) != 0):
        continue
    else:
        row_data = {   
        'ozon_ID': item['dimensions'][0]["id"],
        'Направление': gl.loc[gl["name_short"] == result]["great_category"].values[0],
        'Категория': gl.loc[gl["name_short"] == result]["category"].values[0],
        'sku': gl.loc[gl["name_short"] == result]["sku"].values[0],
        'Название товара': item['dimensions'][0]["name"],
        'session_view_search': item["metrics"][0],
        'ordered_units': item["metrics"][4],
        'revenue': item["metrics"][7],    
        'position_category': item["metrics"][8]
        }
    
    df = pd.concat([df, pd.DataFrame([row_data], columns=tableColumns)], ignore_index=True)

# Распечатаем результат
print(df)

        ozon_ID Направление     Категория        sku  \
0    1256401757      Beauty  Бьюти наборы  768984203   
1    1256348839      Beauty  Бьюти наборы  768982257   
2    1256311449       Clean  Clean_Наборы  923472107   
3    1255377853       Clean  Clean_Наборы  923472107   
4    1255354798       Clean  Clean_Наборы  923472107   
..          ...         ...           ...        ...   
368  1186368665       Clean  Clean_Наборы  923472107   
369  1186170357      Beauty   Жидкое мыло  723245843   
370  1185041318      Beauty  Бьюти наборы  768984203   
371  1184606999        Home  Кондиционеры  723199033   
372  1183514022      Beauty   Жидкое мыло  722019918   

                                       Название товара session_view_search  \
0    Набор натуральный уход для волос "Интенсивное ...                  23   
1    Натуральный уход для волос "Максимальное питан...                  15   
2    Synergetic Набор для уборки Чистота и блеск, а...                  78   
3    Synergetic

Создадим таблицу в нашей БД из DataFrame. Для этого нам необходимы:

<code>pip install sqlalchemy</code>

<code>pip install mysql-connector-python</code>

Единственная проблема - это специальный символ в нашем пароле __@__ для решения этой проблемы мы воспользуемся методами <a href="https://stackoverflow.com/questions/58661569/password-with-cant-connect-the-database">urllib</a>

In [9]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine

# Данные для подключения
user = "main"
password = "P@ssw0rd"
host = "95.163.231.112"
port = 3306
database = "testDB"

# Кодирование пароля
encoded_password = quote_plus(password)

# Подключене
connection_string = f"mysql+mysqlconnector://{user}:{encoded_password}@{host}:{port}/{database}"
engine = create_engine(connection_string, echo=True)

try:
    # Попытка подключения
    connection = engine.connect()
    print("Успешное подключение!")
    df.to_sql("Main_Table", con=engine, if_exists="replace", index=False)
    connection.close()
except Exception as e:
    print(f"Ошибка подключения: {e}")

2023-11-13 13:22:12,386 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-11-13 13:22:12,388 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 13:22:12,400 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-11-13 13:22:12,400 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 13:22:12,407 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-11-13 13:22:12,407 INFO sqlalchemy.engine.Engine [raw sql] {}
Успешное подключение!
2023-11-13 13:22:12,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 13:22:12,442 INFO sqlalchemy.engine.Engine DESCRIBE `testDB`.`Main_Table`
2023-11-13 13:22:12,443 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 13:22:12,453 INFO sqlalchemy.engine.Engine DESCRIBE `testDB`.`Main_Table`
2023-11-13 13:22:12,455 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 13:22:12,461 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `testDB`
2023-11-13 13:22:12,461 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 13:22:12,

<hr>

# __Работа с запросами SQL__

<hr>

Теперь мы можем приступить к работе с базой данных. Для подключения используйте __ip = 95.163.231.112, Username = main, port = 3306, password = Passw0rd.__
![Подключение к серверу](server.png)

Выведем нашу таблицу целиком таблицу.

<code>SELECT *
FROM Main_Table;</code>

![Вся таблица](Images/Table.png)


__Ответим на некоторые вопросы:__

__Какой товар принес больше всего выручки?__

<code>SELECT sku, `Название товара`, SUM(revenue) AS total_revenue
FROM Main_Table
GROUP BY sku, `Название товара`
ORDER BY total_revenue DESC
LIMIT 1;</code>

![1](Images/1.png)

__Какой товар находится выше всего в поиске?__

<code>SELECT sku, MAX(`Название товара`) AS `Название товара`, SUM(position_category) AS total_position_search
FROM Main_Table
GROUP BY sku
ORDER BY total_position_search DESC
LIMIT 1;</code>

![2](Images/2.png)

__Какая товарная категория принесла наибольшую выручку?__

<code>SELECT `Категория`, SUM(revenue) AS total_revenue_by_category
FROM Main_Table
GROUP BY `Категория`
ORDER BY total_revenue_by_category DESC
LIMIT 1;</code>

![3](Images/3.png)

__Товары из какой категории больше всего заказывают?__

<code>SELECT sku, `Категория`, MAX(`Название товара`) AS `Название товара`, MAX(ordered_units) AS orders
FROM Main_Table
GROUP BY `Категория`, sku
ORDER BY orders DESC
LIMIT 1;</code>

![4](Images/4.png)

__Мы хотим проследить дельту изменения выручки в категории по отношению к прошедшему дню. Как это можно сделать?__

Так как у нас нет данных по дням, мы не можем ответить на этот вопрос. Мы можем только оставить значения какие они есть, считая, что это первый день, тогда дельта = revenue - 0;

Будь у нас значение дат report_date, то запрос выглядел как-то так: 

<code>SELECT 
    sku,
    `Категория`,
    COALESCE(
        revenue - (
            SELECT revenue 
            FROM Main_Table as Main_Table_yesterday  
            WHERE Main_Table_yesterday.sku = Main_Table.sku 
              AND Main_Table_yesterday.dates = CURDATE() - INTERVAL 1 DAY
            LIMIT 1
        ), 
        revenue) as delta
FROM Main_Table
GROUP BY sku, `Категория`;
</code>

А в нашем случае дельта будет:

<code>SELECT sku, `Категория`, revenue-0 as delta
FROM Main_Table
GROUP BY sku, `Категория`, delta;
</code>

![5](Images/5.png)


__В какие дни недели товары чаще всего заказывают?__

На этот вопрос без конкретных дат мы тоже ответить не можем. Но если бы они были, то запрос выглядел бы как-то так:

<code>SELECT sku, `Название товара`, DAYOFWEEK(dates) as day_of_week, SUM(revenue) as rev
FROM Main_Table
GROUP BY sku, `Название товара`, DAYOFWEEK(dates);
</code>

<hr>